# Set Up

In [ ]:
# Install python-mne
!pip install mne

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import mne
import numpy as np
import scipy.io
from matplotlib import pyplot as plt
from scipy.io import loadmat
import math as m
from scipy.interpolate import griddata

In [ ]:
# Environment setup
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

# Initialise Variables

In [ ]:
VideoNumber = 20
sfreq = 128
Npoints = sfreq * 60
startPoint = Npoints * (VideoNumber - 1)
endPoint = Npoints * VideoNumber
OriginalNumberofChannels = 48
NumberOfEEGChannel = 32
lengthOfTopographicMap = 40

# Function Definitions

In [ ]:
# Extract EEG only from video
def ExtractEEGonlyFromVideo(idx_starts, transposedDataset, NofChannel):
    values = np.arange(0, 40).tolist()
    Sfreq = sfreq
    Npoints = 60 * Sfreq
    Actualvideodata = []

    for i in values:
        videostartindex = idx_starts[i]
        totalvideolength = transposedDataset[videostartindex:videostartindex + Npoints]
        Actualvideodata.append(totalvideolength)

    Actualvideodata = np.array(Actualvideodata)
    Actualvideodata = Actualvideodata.reshape(-1, NofChannel)
    print(Actualvideodata.shape)

    return Actualvideodata

# Read dataset
def readDataset(verbose=False):
    raw = mne.io.read_raw_bdf('location_of_bdf_file', preload=True)
    ch_names = ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'Status']
    Sfreq = 128
    Npoints = 60 * Sfreq
    rawDataset = raw.copy()
    rawDataset.resample(sfreq=sfreq)

    statusChan = rawDataset._data[47]
    sampleRate = int(rawDataset.info['sfreq'])
    min1 = sampleRate * 60
    idx_starts = []
    i = 0
    while i <= statusChan.shape[0] - min1:
        if statusChan[i] == 4:
            idx_starts.append(i)
            i = i + min1
        else:
            i += 1

    rawDataset32 = rawDataset.copy().drop_channels(ch_names, on_missing='raise')
    channelNames = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7',
                    'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2',
                    'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz']

    rawDataset48 = rawDataset.copy().filter(0.1, 50., fir_design='firwin')
    rawDataset32.filter(0.1, 50., fir_design='firwin')
    rawDatasetReReferenced32 = rawDataset32.copy().set_eeg_reference(ref_channels='average')
    rawDatasetForMontageLocation = rawDatasetReReferenced32.copy()
    rawDatasetReReferenced48 = rawDataset48.copy().set_eeg_reference(ref_channels='average')

    transposedDataset48 = np.transpose(rawDataset48._data)
    transposedDataset32 = np.transpose(rawDatasetReReferenced32._data)

    Actualvideodata48 = ExtractEEGonlyFromVideo(idx_starts, transposedDataset48, OriginalNumberofChannels)
    Actualvideodata32 = ExtractEEGonlyFromVideo(idx_starts, transposedDataset32, NumberOfEEGChannel)

    return Actualvideodata32, Actualvideodata48, rawDatasetForMontageLocation

# Get channel names
def getChannelNames():
    channelNames1 = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 'CP1', 'CP5', 'P7',
                     'P3', 'Pz', 'PO3', 'O1', 'Oz', 'O2', 'PO4', 'P4', 'P8', 'CP6', 'CP2',
                     'C4', 'T8', 'FC6', 'FC2', 'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz']
    return channelNames1

# Get channel info for sample
def getChannellInfoForSample(channelNames1, channelValues, onlyValues=False):
    channelValuesforCurrentSample = []
    for i, ch in enumerate(channelNames1):
        chValue = channelValues[i]
        if onlyValues:
            channelValuesforCurrentSample.append(chValue)
        else:
            channelValuesforCurrentSample.append((ch, chValue))
    return channelValuesforCurrentSample

# Convert Cartesian to Spherical coordinates
def cart2sph(x, y, z):
    x2_y2 = x**2 + y**2
    r = m.sqrt(x2_y2 + z**2)
    elev = m.atan2(z, m.sqrt(x2_y2))
    az = m.atan2(y, x)
    return r, elev, az

# Convert Polar to Cartesian coordinates
def pol2cart(theta, rho):
    return rho * m.cos(theta), rho * m.sin(theta)

# Azimuthal Equidistant Projection
def azim_proj(pos):
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, m.pi / 2 - elev)

# Get 3D coordinates
def get3DCoordinates(MontageChannelLocation, EEGChannels):
    MontageChannelLocation = MontageChannelLocation[-EEGChannels:]
    location = []
    for i in range(32):
        v = list(MontageChannelLocation[i].values())
        location.append(v[1] * 1000)
    MontageLocation = np.round(np.array(location), 1).tolist()
    return MontageLocation

# Convert 3D to 2D
def convert3DTo2D(pos_3d):
    pos_2d = [azim_proj(e) for e in pos_3d]
    return pos_2d

# Get matrix indexes from 2D positions
def getMatrixIndexesFrom2DPositions(xPos, yPos, minX, minY, maxX, maxY, numChannels, CordinateYellowRegion, verbose=False):
    x = ((xPos - minX) * (numChannels - 1)) / (maxX - minX)
    y = ((yPos - minY) * (numChannels - 1)) / (maxY - minY)
    indexX = round(x)
    indexY = round(y)

    if [indexX, indexY] not in CordinateYellowRegion.tolist():
        indexX1, indexY1 = indexX, indexY
    elif ([indexX, indexY + 1] not in CordinateYellowRegion.tolist()) and ([indexX, indexY + 1] != [indexX, lengthOfTopographicMap]):
        indexX1, indexY1 = indexX, indexY + 1
    elif [indexX, indexY - 1] not in CordinateYellowRegion.tolist():
        indexX1, indexY1 = indexX, indexY - 1
    elif ([indexX - 1, indexY] not in CordinateYellowRegion.tolist()) and ([indexX - 1, indexY] != [-1, indexY]):
        indexX1, indexY1 = indexX - 1, indexY
    else:
        indexX1, indexY1 = indexX + 1, indexY

    if verbose:
        print(f"Transformed positions: {x}, {y} - Generated matrix indexes: {indexX}, {indexY}")

    return indexX1, indexY1

# Get 2D topographic map channel indexes
def get2DTopographicMapChannelIndexes(lengthOfTopographicMap, CordinateYellowRegion):
    pos3D = get3DCoordinates(MontageChannelLocation, NumberOfEEGChannel)
    pos2D = convert3DTo2D(pos3D)

    minX = min(pos2D, key=lambda t: t[0])[0]
    maxX = max(pos2D, key=lambda t: t[0])[0]
    minY = min(pos2D, key=lambda t: t[1])[1]
    maxY = max(pos2D, key=lambda t: t[1])[1]

    coordinates2dWithoutScale = []
    for ch in pos2D:
        x, y = getMatrixIndexesFrom2DPositions(ch[0], ch[1], minX, minY, maxX, maxY, lengthOfTopographicMap, CordinateYellowRegion)
        coordinates2dWithoutScale.append((x, y))

    return coordinates2dWithoutScale

# Visualization of 2D map
def visualize2DMap(channelValues):
    channelNames1 = getChannelNames()
    pos2d = get2DTopographicMapChannelIndexes(lengthOfTopographicMap, CordinateYellowRegion)
    Topomap = np.zeros((40, 40))
    channelInfo = getChannellInfoForSample(channelNames1, channelValues)

    for (pos, info) in zip(pos2d, channelInfo):
        Topomap[pos[0], pos[1]] = info[1]

    return Topomap

# Generate a dataset with a 2D topographic map and output values
def generateDatasetWith2DTopographicMapAndOutputValues():
    channelValues48 = readDataset(verbose=False)[1]
    channelNames1 = getChannelNames()

    newDataset = []
    for sample in channelValues48:
        channelValues32 = getChannellInfoForSample(channelNames1, sample, onlyValues=True)
        newSample = visualize2DMap(channelValues32)
        newDataset.append(newSample)

    newDataset = np.array(newDataset)
    return newDataset

# Generate and Save Topographic Maps

In [ ]:
# Initialise grid
CordinateYellowRegion = np.array(np.where(np.zeros((40, 40)))).T
MontageChannelLocation = mne.channels.read_montage('standard_1020').get_positions()['ch_pos']
lengthOfTopographicMap = 40

# Generate and save the dataset
topographicMapDataset = generateDatasetWith2DTopographicMapAndOutputValues()
np.save('location_to_save_npy_file/topographicMapDataset.npy', topographicMapDataset)

# Print shape
print(f"Generated dataset shape: {topographicMapDataset.shape}")